# Setting up a simple authentication

In this section we will set up a simple PAM configuration to allow certificate authentication for using the yubikey. 

_The trend currently is to start to use `sssd`, unfortunately not all distrobutions will offer the 2.x version of SSSD that allows for local certificate authentication (Debian buster provide 1.16 version). For that reason we will use the older (no longer maintained) project `pam_pkcs11`. I believe the use of `yubico-pam` module could also do similar functionality, but for our purposes we will use the `pam_pkcs11` module which I am more familiar with._

In Debian the `/etc/pam_pkcs11/pam_pkcs11.conf` is not part of the default load, using the example file to create the configuration can be found `/usr/share/doc/libpam-pkcs11/examples/pam_pkcs11.conf.example.gz`. For our purposes we will look at the configuration I have prepared for this example.



First let's make the user for testing.

In [1]:
%%bash
sudo adduser --gecos "Rachel Andomsoul" --disabled-password randomsoul

Adding user `randomsoul' ...
Adding new group `randomsoul' (1001) ...
Adding new user `randomsoul' (1001) with group `randomsoul' ...
Creating home directory `/home/randomsoul' ...
Copying files from `/etc/skel' ...


Now we can set up the `pam_pkcs11.conf` file and select it for using the opensc section. 

Some important configuration details: 

```
use_pkcs11_module = opensc;

pkcs11_module opensc {
  module = /usr/lib/x86_64-linux-gnu/opensc-pkcs11.so;
  description = "OpenSC PKCS#11 module";
  ca_dir = /etc/pam_pkcs11/cacerts;
  crl_dir = /etc/pam_pkcs11/crls;
  cert_policy = ca, signature;
}

use_mappers = generic, null;

mapper generic {
  debug = true;
  module = internal;
  ignorecase = false;
  cert_item  = cn;
  mapfile = file:///etc/pam_pkcs11/generic_mapping;
  use_getpwent = false;
}
```

The choice in using generic because of some issues found with cn and uid internal mappers not always working, likely it is something simple. However, the use of the generic mapping mechanism does transition well to the `sssd` local certificate method in which use of a regex style subject mapping to local user is use instead of the internal mapping mechanisms like cn and uid.

The `/etc/pam_pkcs11/generic_mapping` file contains a line for the test user: 

```
Rachel Andomsoul -> randomsoul
```

This configuration will use the openssl style certificates with hash reference to provide the authentication matching.

In [2]:
%%bash
sudo cp certificates/ca-root.pem /etc/pam_pkcs11/cacerts
sudo openssl rehash /etc/pam_pkcs11/cacerts

In [3]:
%%bash
ls -l /etc/pam_pkcs11/cacerts

total 4
-rw-r--r-- 1 root root 1411 Oct 29 14:01 ca-root.pem
lrwxrwxrwx 1 root root   11 Oct 29 14:01 d5d10f00.0 -> ca-root.pem


In the `/etc/pam.d/common-auth` file we will add the following: 

```
auth  [success=ok ignore=ignore default=die]  pam_pkcs11.so wait_for_card
```

## Using NSS3 and pam_pkcs11